## Environment Setup

In [ ]:
%pip install -q pyreadstat gdown xgboost shap

In [ ]:
%pip install scikit-learn==1.4.2 tensorflow==2.15
# Restart the session after running this cell

In [ ]:
%pip install -U gdown

In [ ]:
%pip install -q ydata_profiling

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, RocCurveDisplay, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import BayesianRidge
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import shap

## Data Downloading

In [ ]:
import gdown
# All
gdown.download(id="1S8agHpzNckOElaGqRU7byP40tPIznxXW")
# Zinc, Copper
gdown.download(id="1ausKJoiSEUVg1hxTumwQAeAMfgAZrRou")

In [ ]:
df_raw = pd.read_spss("./HTN_NEW.sav").drop(["HTN_2_1"], axis=1)
df_raw.head()

In [ ]:
df_zink = pd.read_spss("./HTN_zink.sav")
df_zink.head()

In [ ]:
df_raw.dtypes, df_zink.dtypes

In [ ]:
df_zink = df_zink.set_index('FileNo')

In [ ]:
df = df_raw.join(df_zink, on="File.No", how="left")
df

In [ ]:
df['File.No'].value_counts()

In [ ]:
# Remove duplicated indecies
df = df.drop(index=df.index[df['File.No'] == 2586])
df = df.drop(index=df.index[df['File.No'] == 2012])

## Profiling

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="HTN Profiling Report")
# profile.to_file("your_report.html")

## Preprocessing

In [ ]:
df.shape

In [ ]:
renaming_mapper = {
    'HTN_phase1': 'HTN_2_1',
}
df = df.rename(columns=renaming_mapper)

In [ ]:
df.HTN_2_1.value_counts()

In [ ]:
df.smokingstutus.value_counts()

In [ ]:
df.Diabete.value_counts()

In [ ]:
def show(df_):
    print(f"ratio: {round(df_.shape[0] / df.shape[0] * 100,1)}%")
    return df_.head(3)

In [ ]:
def map_diabete(x):
    match x:
        case "<126":
            return 0.0
        case ">=126":
            return 1.0
        case _:
            return x;

def map_sex(x):
    match x:
        case "male":
            return 0.0
        case "female":
            return 1.0
        case _:
            # print(f"some thing wrong {x=}") ???????? only 2
            return x


target_names = ["HTN-", "HTN+"]


def map_htn(x):
    if x == target_names[0]:
        return 0.0
    if x == target_names[1]:
        return 1.0
    return x


df.Diabete = df.Diabete.apply(map_diabete)
df.Sex = df.Sex.apply(map_sex)
df.HTN_2_1 = df.HTN_2_1.apply(map_htn)

In [ ]:
df = df.drop(["HTN_phase2", "File.No", 'phase1'], axis=1)

In [ ]:
# Encode Categorical
oneHotEncoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encodedSmokingStatus = oneHotEncoder.fit_transform(df[['smokingstutus']])
encodedSmokingStatusDF = pd.DataFrame(encodedSmokingStatus, columns=oneHotEncoder.get_feature_names_out())

df = pd.concat([df, encodedSmokingStatusDF], axis=1)

In [ ]:
df = df.drop(['smokingstutus'], axis=1)

In [ ]:
n_na = df.apply(
    lambda x: np.where(x[1:].isna())[0].shape[0], axis=1
)
n_na.describe(percentiles=[0.75, 0.9, 0.93, 0.95])

In [ ]:
df

In [ ]:
renaming_mapper = {
    'MLP': 'MLR',
    'MPP': 'MPR',
    'LYM_Sh': 'LYM',
    'NEUT_Sh': 'NEUT',
    'hs_CRP': 'hsCRP',
    'age_1': 'age'
}
df = df.rename(columns=renaming_mapper)

In [ ]:
df_pure = df[n_na <= 3]
df_pure[~df_pure.HTN_2_1.isna()].shape[0], df[~df.HTN_2_1.isna()].shape[0]

In [ ]:
y_ = df_pure.HTN_2_1
X_ = df_pure.drop(["HTN_2_1"], axis=1)

In [ ]:
columns = X_.columns
without_sex_columns = columns.drop(['Sex'])

In [ ]:
percentile_df = X_.describe(percentiles=[.01,.99]).transpose()
percentile_df

In [ ]:
X_.shape, y_.shape

In [ ]:
# Remove records that have `hsCRP` above 10
hsCRP_filter = X_[X_['hsCRP'] > 10].index
X_ = X_.drop(hsCRP_filter, axis=0)
y_ = y_.drop(hsCRP_filter, axis=0)
df_pure = df_pure.drop(hsCRP_filter, axis=0)

In [ ]:
for c in percentile_df.index:
  one_pecent = percentile_df.loc[c]['1%']
  ninty_precent = percentile_df.loc[c]['99%']

  if (c == 'hsCRP'):
    # Ignore the upper ninty percent part of the hsCRP since we removed them already
    filter = (X_[c] < one_pecent)
  else:
    filter = (X_[c] < one_pecent) | (X_[c] > ninty_precent)

  X_.loc[filter, c] = np.nan

In [ ]:
cat_columns_names = ['Diabete', 'Sex', 'smokingstutus_Ex-smoker', 'smokingstutus_current smoker', 'smokingstutus_non smoker']
cat_columns_count = len(cat_columns_names)
cat_columns_indices = [X_.columns.get_loc(c) for c in cat_columns_names]
cat_columns_indices

In [ ]:
non_cat_columns_indices = [i for i in range(0, len(X_.columns)) if i not in cat_columns_indices]

# Change the order of columns
cat_columns = X_.columns[cat_columns_indices]
non_cat_columns = X_.columns[non_cat_columns_indices]

columns = [*non_cat_columns.tolist(), *cat_columns.tolist()]

X_ = X_[columns]
X_

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_, y_, test_size=0.1, random_state=42
)

In [ ]:
# KNN Imputer
imputer = KNNImputer(n_neighbors=5, weights="distance")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Round values since they're categorical
X_train[:, -5:] = np.round(X_train[:,  -5:])
X_test[:, -5:] = np.round(X_test[:, -5:])

In [ ]:
# Remove NA Targets

train_na_idx = y_train.isna()
X_train = X_train[~train_na_idx]
y_train = y_train[~train_na_idx]

test_na_idx = y_test.isna()
X_test = X_test[~test_na_idx]
y_test = y_test[~test_na_idx]

In [ ]:
# Standard Scaler
scaler = StandardScaler()

X_train_standard = scaler.fit_transform(X_train[:, :-5])
X_test_standard = scaler.transform(X_test[:, :-5])

In [ ]:
# Concat all non-scaled columns with scaled ones
X_train_cat = X_train[:, -5:]
X_train = np.concatenate([X_train_standard, X_train_cat], axis=1)

X_test_cat = X_test[:, -5:]
X_test = np.concatenate([X_test_standard, X_test_cat], axis=1)

In [ ]:
# Convert them to Dataframe again
X_train = pd.DataFrame(X_train, columns=columns).drop(["Zinc_Copper", "copper_zinc"], axis=1)
X_test = pd.DataFrame(X_test, columns=columns).drop(["Zinc_Copper", "copper_zinc"], axis=1)

In [ ]:
columns.remove("Zinc_Copper")
columns.remove("copper_zinc")

In [ ]:
X_train.shape, X_test.shape

In [ ]:
renaming_dictionary = {
    "hsCRP": "hs-CRP",
    "Glocuse": "glucose",
    "LdL": 'LDL',
    "Cholesterol": "cholesterol",
    "Triglycerides": "triglycerides",
    "Uricacid": "uric acid",
    "bunincerate": "bun incerate",
    "uriaid_HDL": "uric acid to HDL",
    "Uricacid_bunincerate": "uric acid bun incerate",
    "AnxietyScore": "anxiety score",
    "Diabete": "diabetes",
    "DepressionScore": "depression score",
    "Copper": "copper",
    "Zinc": "zinc",
    "Sex": "sex",
    "smokingstutus_Ex-smoker": "smoking status: ex-smoker",
    "smokingstutus_current smoker": "smoking status: current-smoker",
    "smokingstutus_non smoker": "smoking stutus: non-smoker"
}

# Final renaming for make the names correct
X_train = X_train.rename(renaming_dictionary, axis=1)
X_test = X_test.rename(renaming_dictionary, axis=1)

In [ ]:
# save data for regresstion analysis

Xy_train = X_train.copy()
Xy_train["HTN_2_1"] = y_train.to_numpy()
Xy_train.to_csv("htn_xy_train.csv")

Xy_test = X_test.copy()
Xy_test["HTN_2_1"] = y_test.to_numpy()
Xy_test.to_csv("htn_xy_test.csv")

## Utils

In [ ]:
all_data_label = ""

In [ ]:
def get_best_threshold(y_true, y_pred):
  fpr, tpr, thresholds = roc_curve(y_true, y_pred)
  max_index = np.argmax(tpr + (1 - fpr))
  return thresholds[max_index]

In [ ]:
def showConfusionMatrix(y_true, y_pred, name, display_labels=None,
                        figsize=(4, 3), dpi=400,
                        title_fontsize=24, label_fontsize=20,
                        tick_rotation=0,
                        values_format=None, normalize=None):
    """
    Colab-optimized confusion matrix visualization.
    """
    # Create figure and axes first
    fig, ax = plt.subplots(figsize=(12, 16), dpi=dpi, facecolor='white')

    # Generate confusion matrix display
    disp = ConfusionMatrixDisplay.from_predictions(
        y_true, y_pred,
        display_labels=display_labels,
        normalize=normalize,
        xticks_rotation=tick_rotation,
        values_format=values_format,
        ax=ax,  # Explicitly specify the axes
        colorbar=False,
        include_values=True
    )


    for text in disp.text_.ravel():
        text.set_fontsize(28)

    # Customize appearance
    disp.ax_.set_title(name, fontsize=title_fontsize, pad=20)
    disp.ax_.set_xlabel('Predicted Label', fontsize=label_fontsize)
    disp.ax_.set_ylabel('True Label', fontsize=label_fontsize)
    disp.ax_.tick_params(axis='both', labelsize=label_fontsize-2)

    # Add colorbar with colab-friendly settings
    cbar = fig.colorbar(disp.im_, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=label_fontsize-2)


    # Colab-specific layout adjustments
    plt.tight_layout()
    plt.show()
    plt.close(fig)  # Clear memory after display

def generate_single_report(X, y_true, y_pred, y_pred_prob = None, name = ""):
    print(f"#================={name}=================#")
    print(classification_report(y_true, y_pred, target_names=target_names))

    if y_pred_prob is not None:
        roc_value = roc_auc_score(y_true, y_pred_prob)
        print(f"\nROC AUC: {roc_value}")

    showConfusionMatrix(y_true, y_pred, name)

    return y_pred

def report(estimator, X_train, X_test, y_pred_prob_f=None, name="", y_train=y_train, y_test=y_test):
    y_pred_prob_train = y_pred_prob_f(X_train) if y_pred_prob_f is not None else None
    y_pred_prob_test = y_pred_prob_f(X_test) if y_pred_prob_f is not None else None

    best_threshold = get_best_threshold(y_train, y_pred_prob_train) if y_pred_prob_train is not None else 0.5

    y_pred_train = estimator(X_train)
    y_pred_test = estimator(X_test)
    if (np.mod(y_pred_train[0], 1) != 0):
      y_pred_train = np.where(y_pred_train < best_threshold, 0, 1)
      y_pred_test = np.where(y_pred_test < best_threshold, 0, 1)

    if (best_threshold != 0.5):
      print(f"Best Threshold: {best_threshold}")

    generate_single_report(X_train, y_train, y_pred_train, y_pred_prob_train, f"{name} - Train")
    generate_single_report(X_test, y_test, y_pred_test,y_pred_prob_test, f"{name} - Test")
    print("\n#==============================================================#")

In [ ]:
# shap_value should be type of `Explainer`
def plotBar(shap_value, with_cohort=True, last_index=len(X_test)):
  fig, ax = plt.subplots(figsize=(10, 14), dpi=400, facecolor='white')
  if (with_cohort):
    cohort = np.where(X_test[:last_index].Sex == 1, "Female", "Male" )
    shap.plots.bar(shap_value.cohorts(cohort).abs.mean(0), max_display=18, ax=ax)
  else:
    shap.plots.bar(shap_value, max_display=18, ax=ax)

In [ ]:
def get_correct_df(estimator, X, y):
  y_pred = estimator(X)

  # Check if the predictions are probabilites by default
  if (np.mod(y_pred[0], 1) != 0):
    y_pred = np.round(y_pred)

  correct_idx = y_pred == y
  return X[correct_idx.values], y[correct_idx.values]

In [ ]:
def get_ann_shap(model, X_train, y_train, X_test, y_test, last_index=250):
  ann_estimator = lambda x: model.predict(x)[:, 0]
  ann_explainer = shap.explainers.Permutation(model.predict, X_train)
  X_test_correct, y_test_correct = get_correct_df(ann_estimator, X_test, y_test)
  ann_shap = ann_explainer(X_test_correct[:last_index])

  return ann_shap, X_test_correct, y_test_correct

In [ ]:
import copy

def filter_shap_by_column(shap_value, column_index, condition_value, condition_type='equal'):
  modified_shap = copy.deepcopy(shap_value)

  if (condition_type == 'equal'):
    condition = modified_shap.data[:, column_index] == condition_value

  modified_shap.values = modified_shap.values[condition]
  modified_shap.data = modified_shap.data[condition]
  modified_shap.base_values = modified_shap.base_values[condition]

  # Drop column_index column
  modified_shap.values = np.concatenate([modified_shap.values[:, :column_index], modified_shap.values[:, column_index + 1:]], axis=1)
  modified_shap.data = np.concatenate([modified_shap.data[:, :column_index], modified_shap.data[:, column_index + 1:]], axis=1)

  return modified_shap

In [ ]:
def filter_pair_df_by_column(X, y, column_name, condition_value, condition_type='equal'):
  condition = X[column_name] == condition_value
  return X.loc[condition.values], y.loc[condition.values]

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def show_shap_bar(values):
    fig, ax = plt.subplots(figsize=(10, 12), dpi=400, facecolor='white')
    shap.plots.bar(values, max_display=15, ax=ax)

def show_shap_beeswarm(values):
    shap.plots.beeswarm(values, max_display=15, plot_size=(12, 10))

In [ ]:
X_test_male, y_test_male = filter_pair_df_by_column(X_test, y_test, 'sex', 0)
X_test_female, y_test_female = filter_pair_df_by_column(X_test, y_test, 'sex', 1)

X_train_male, y_train_male = filter_pair_df_by_column(X_train, y_train, 'sex', 0)
X_train_female, y_train_female = filter_pair_df_by_column(X_train, y_train, 'sex', 1)

sex_column_index = X_train.columns.get_loc('sex')

## Models

### BayesianRidge

In [ ]:
def br_runner(X_train, y_train, X_test, name):
  param_grid = {
      "n_iter": [10, 20, 30, 50, 100, 150, 300, 600],
  }
  grid = GridSearchCV(BayesianRidge(), param_grid, cv=5).fit(X_train, y_train)

  if hasattr(grid, 'best_params_'):
    print(grid.best_params_)

  report(grid.predict, X_train, X_test, lambda X: grid.predict(X))

  model = grid.best_estimator_

  model_explainer = shap.Explainer(model, X_train)
  X_test_correct, y_test_correct = get_correct_df(model.predict, X_test, y_test)
  model_shap = model_explainer(X_test_correct)

  return model, model_explainer, model_shap, X_test_correct, y_test_correct, grid

#### Performance Report

In [ ]:
br_model, br_explainer, br_shap, br_X_test_correct, br_y_test_correct, br_grid  = br_runner(X_train, y_train, X_test, f"BR - {all_data_label}")

#### Shap Bee Swarm

In [ ]:
show_shap_beeswarm(br_shap)

### KNN

In [ ]:
def knn_runner(X_train, y_train, X_test, name):
  param_grid = {
      "n_neighbors": [8, 14, 18, 22],
  }
  grid = GridSearchCV(KNeighborsClassifier(weights="uniform"), param_grid, cv=5).fit(X_train, y_train)

  if hasattr(grid, 'best_params_'):
    print(grid.best_params_)

  report(grid.predict, X_train, X_test, lambda X: grid.predict_proba(X)[:, 1], name)

  model = grid.best_estimator_

  model_explainer = shap.KernelExplainer(model.predict_proba, shap.kmeans(X_train, 30))
  X_test_correct, y_test_correct = get_correct_df(model.predict, X_test, y_test)
  model_shap = model_explainer(X_test_correct)


  return model, model_explainer, model_shap, X_test_correct, y_test_correct, grid

##### Performance Report

In [ ]:
knn_model, knn_explainer, knn_shap, knn_X_test_correct, knn_y_test_correct, knn_grid  = knn_runner(X_train, y_train, X_test, f"KNN")

##### Shap Bar

In [ ]:
show_shap_bar(knn_shap[:, :, 1])

##### Shap Beeswarm

In [ ]:
show_shap_beeswarm(knn_shap[:, :, 1])

### Logistic Regression

In [ ]:
def lr_runner(X_train, y_train, X_test, y_test, name):
  model = LogisticRegressionCV(cv=5, random_state=42, max_iter=10_00).fit(X_train, y_train)

  if hasattr(model, 'best_params_'):
    print(model.best_params_)
  report(model.predict, X_train, X_test, lambda X: model.predict_proba(X)[:, 1], name)

  feature_importance = model.coef_.flatten()

  model_explainer = shap.Explainer(model, X_train)
  X_test_correct, y_test_correct = get_correct_df(model.predict, X_test, y_test)
  model_shap = model_explainer(X_test_correct)

  return model, model_explainer, model_shap, feature_importance, X_test_correct, y_test_correct

##### Performance Report

In [ ]:
lr_model, lr_explainer, lr_shap, lr_feature_importance, lr_X_test_correct, lr_y_test_correct = lr_runner(X_train, y_train, X_test, y_test, "Logistic Regression")

##### Shap Bar

In [ ]:
show_shap_bar(lr_shap)

##### Shap Beeswarm

In [ ]:
show_shap_beeswarm(lr_shap)

### Random Forest

In [ ]:
def rf_runner(X_train, y_train, X_test, name):
  param_grid = {"n_estimators": [50, 60, 70], "max_depth": [4, 6]}

  class_weight = {0: 1, 1: 2}

  grid = GridSearchCV(RandomForestClassifier(random_state=42, class_weight=class_weight), param_grid, cv=5).fit(
      X_train, y_train
  )

  if hasattr(grid, 'best_params_'):
    print(grid.best_params_)
  report(grid.predict, X_train, X_test, lambda X: grid.predict_proba(X)[:, 1], name)

  model = grid.best_estimator_

  feature_importance = model.feature_importances_

  model_explainer = shap.Explainer(model)
  X_test_correct, y_test_correct = get_correct_df(model.predict, X_test, y_test)
  model_shap = model_explainer(X_test_correct)

  return model, model_explainer, model_shap, feature_importance, X_test_correct, y_test_correct

##### Performance Report

In [ ]:
rf_model, rf_explainer, rf_shap, rf_feature_importance, rf_X_test_correct, rf_y_test_correct  = rf_runner(X_train, y_train, X_test, "Random Forest")

##### Shap Bar

In [ ]:
show_shap_bar(rf_shap[:, :, 1])

##### Shap Beeswarm

In [ ]:
show_shap_beeswarm(rf_shap[:, :, 1])

### XGBoost

In [ ]:
def xgb_runner(X_train, y_train, X_test, name):
  param_grid = {
      "n_estimators": [100],
      "max_depth": [3],
  }

  grid = GridSearchCV(xgb.XGBClassifier(random_state=42, tree_method="approx", scale_pos_weight=2.3), param_grid, cv=3, scoring="f1_weighted").fit(
      X_train, y_train
  )

  if hasattr(grid, 'best_params_'):
    print(grid.best_params_)
  report(grid.predict, X_train, X_test, lambda X: grid.predict_proba(X)[:, 1], name)

  model = grid.best_estimator_
  feature_importance = model.feature_importances_

  model_explainer = shap.Explainer(model, X_train)
  X_test_correct, y_test_correct = get_correct_df(model.predict, X_test, y_test)
  model_shap = model_explainer(X_test_correct)

  return model, model_explainer, model_shap, feature_importance, X_test_correct, y_test_correct, grid

##### Performance Report

In [ ]:
xgb_model, xgb_explainer, xgb_shap, xgb_feature_importance, xgb_X_test_correct, xgb_y_test_correct, xgb_grid  = xgb_runner(X_train, y_train, X_test, "XGBoost")

##### Shap Bar

In [ ]:
show_shap_bar(xgb_shap)

##### Shap Beeswarm

In [ ]:
show_shap_beeswarm(xgb_shap)

### Neural Network

In [ ]:
import tensorflow as tf
def ann_runner(X_train, y_train, X_test, y_test, name):
  tf.keras.utils.set_random_seed(812)

  model = tf.keras.models.Sequential(
      [
          tf.keras.Input(shape=(X_train.shape[1],)),
          tf.keras.layers.Dense(32, activation="leaky_relu"),
          tf.keras.layers.Dense(16, activation="leaky_relu"),
          tf.keras.layers.Dense(4, activation="leaky_relu"),
          tf.keras.layers.Dense(1, activation="sigmoid"),
      ]
  )
  # model = tf.keras.models.Sequential(
  #     [
  #         tf.keras.Input(shape=(X_train.shape[1],)),
  #         tf.keras.layers.Dense(512, activation="leaky_relu"),
  #         tf.keras.layers.Dense(1, activation="sigmoid"),
  #     ]
  # )

  class_weight = {0: 1.,
                1: 1.5}

  earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

  model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0002, weight_decay=0.007), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.AUC()])
  model.fit(X_train, y_train, validation_split=0.2, epochs=1000, batch_size=1024, class_weight=class_weight, callbacks=[earlyStopping], verbose=0)
  print()
  estimator = lambda x: model.predict(x)[:, 0]
  report(estimator, X_train, X_test, estimator, name, y_train=y_train, y_test=y_test)

  return model, estimator

##### Performance Report

In [ ]:
ann_model, ann_estimator = ann_runner(X_train, y_train, X_test, y_test, "Neural Network")

##### Neural Network Structure

In [ ]:
ann_model.summary()

In [ ]:
%%time
if (os.path.exists('ann_shap.pickle')):
  with open(f'ann_shap.pickle', 'rb') as file:
    ann_shap = pickle.load(file)
  with open(f'ann_X_test_correct.pickle', 'rb') as file:
    ann_X_test_correct = pickle.load(file)
  with open(f'ann_y_test_correct.pickle', 'rb') as file:
    ann_y_test_correct = pickle.load(file)
else:
  # I seperate it because it tooks a lot of time to respond!
  # DO NOT run this cell if you don't have at least 7mins time (for 100 record)
  ann_shap, ann_X_test_correct, ann_y_test_correct = get_ann_shap(ann_model, X_train, y_train, X_test, y_test)
  with open(f'ann_shap.pickle', 'wb') as file:
    pickle.dump(ann_shap, file)
  with open(f'ann_X_test_correct.pickle', 'wb') as file:
    pickle.dump(ann_X_test_correct, file)
  with open(f'ann_y_test_correct.pickle', 'wb') as file:
    pickle.dump(ann_y_test_correct, file)

##### Shap Bar

In [ ]:
plotBar(ann_shap, False, 100)

##### Shap Beeswarm

In [ ]:
shap.plots.beeswarm(ann_shap, max_display=18, plot_size=(12, 12))

## Feature Importance

In [ ]:
def show_shap_fi(lr_shap, rf_shap, xgb_shap, ann_shap, columns=columns):
  lr_shap_imp = np.mean(np.abs(lr_shap.values), 0)
  knn_shap_imp = np.mean(np.abs(knn_shap[:, :, 1].values), 0)
  rf_shap_imp = np.mean(np.abs(rf_shap[:, :, 1].values), 0)
  xgb_shap_imp = np.mean(np.abs(xgb_shap.values), 0)
  ann_shap_imp = np.mean(np.abs(ann_shap.values), 0)
  print(">>",ann_shap_imp.shape)

  feature_importances_columns = ['lr_shap_imp', 'knn_shap_imp', 'rf_shap_imp', 'xgb_shap_imp', 'ann_shap_imp']

  shap_imp_df = pd.DataFrame({
      'feature': columns,
      'rf_shap_imp': rf_shap_imp,
      'knn_shap_imp': knn_shap_imp,
      'xgb_shap_imp': xgb_shap_imp,
      'lr_shap_imp': lr_shap_imp,
      'ann_shap_imp': ann_shap_imp,
      })

  shap_imp_df = shap_imp_df.sort_values('xgb_shap_imp', ascending=False)
  shap_imp_df = shap_imp_df.style.format( '{:.4f}', subset=feature_importances_columns).bar(subset=feature_importances_columns, color='#511db8', width=60)

  return shap_imp_df

In [ ]:
def show_normalized_shap_fi(lr_shap, rf_shap, xgb_shap, ann_shap, columns=columns):
  lr_shap_imp = np.mean(np.abs(lr_shap.values), 0)
  rf_shap_imp = np.mean(np.abs(rf_shap[:, :, 1].values), 0)
  xgb_shap_imp = np.mean(np.abs(xgb_shap.values), 0)
  ann_shap_imp = np.mean(np.abs(ann_shap.values), 0)

  lr_shap_prob = lr_shap_imp / np.sum(lr_shap_imp) * 100
  rf_shap_prob = rf_shap_imp / np.sum(rf_shap_imp) * 100
  xgb_shap_prob = xgb_shap_imp / np.sum(xgb_shap_imp) * 100
  ann_shap_prob = ann_shap_imp / np.sum(ann_shap_imp) * 100

  feature_importances_columns = ['lr_shap_prob', 'rf_shap_prob', 'xgb_shap_prob', 'ann_shap_prob']

  shap_imp_df = pd.DataFrame({
      'feature': columns,
      'rf_shap_prob': rf_shap_prob,
      'xgb_shap_prob': xgb_shap_prob,
      'lr_shap_prob': lr_shap_prob,
      'ann_shap_prob': ann_shap_prob,
      })

  shap_imp_df = shap_imp_df.sort_values('ann_shap_prob', ascending=False)
  shap_imp_df = shap_imp_df.style.format( '{:.2f}', subset=feature_importances_columns).bar(subset=feature_importances_columns, color='#511db8', width=60)

  return shap_imp_df

#### Shap Feature Importance

In [ ]:
show_shap_fi(lr_shap, rf_shap, xgb_shap, ann_shap)

In [ ]:
# Throws error: All arrays must be of the same length
# show_shap_fi(lr_pure_shap, rf_pure_shap[:, :, 1], xgb_pure_shap)

#### Shap Normalized Feature Importance (Probability Distribution %)

In [ ]:
show_normalized_shap_fi(lr_shap, rf_shap, xgb_shap, ann_shap)

#### Legacy Feature Importance

We don't have Neural Network Feature Importance in this method.

In [ ]:
def show_fi(lr_fi, rf_fi, xgb_fi):
  feature_importances_columns = ['lr_feature_importance', 'rf_feature_importance', 'xgb_feature_importance']

  fr_df = pd.DataFrame({
      'feature': columns,
      'lr_feature_importance': lr_fi,
      'rf_feature_importance': rf_fi,
      'xgb_feature_importance': xgb_fi
      })

  fr_df = fr_df.sort_values('xgb_feature_importance', ascending=False)
  fr_df = fr_df.style.format( '{:.4f}', subset=feature_importances_columns).bar(subset=feature_importances_columns, color='#511db8', width=60)

  return fr_df

In [ ]:
show_fi(lr_feature_importance, rf_feature_importance, xgb_feature_importance)